# MMfem Interactive Notebook

Interfaz interactiva para resolver problemas de Navier-Stokes sin escribir código.

## Instrucciones

1. Ejecuta todas las celdas en orden (Cell > Run All)
2. Usa los widgets interactivos para configurar tu problema
3. Presiona "Resolver" para ejecutar la simulación
4. Visualiza los resultados

In [2]:
# Imports
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from ngsolve import CF, Draw
import matplotlib.pyplot as plt
%matplotlib inline

# Import MMfem
try:
    from mmfem import (
        rectangle_mesh, L_mesh,
        #taylor_hood,
        #picard_iteration, newton_iteration,
        #adaptive_solve,
        #generate_convergence_plot
    )
    print("✓ MMfem importado correctamente")
except ImportError as e:
    print(f"❌ Error importando MMfem: {e}")
    print("Asegúrate de haber instalado MMfem: pip install -e .")

✓ MMfem importado correctamente


## Configuración del Problema

In [3]:
# Create interactive widgets
style = {'description_width': '200px'}
layout = widgets.Layout(width='500px')

# Domain selection
domain_widget = widgets.Dropdown(
    options=[('Rectángulo (cavity flow)', 'rectangle'),
             ('Dominio en L (con singularidad)', 'L')],
    value='rectangle',
    description='Dominio:',
    style=style,
    layout=layout
)

# Mesh size
mesh_size_widget = widgets.FloatSlider(
    value=0.05,
    min=0.01,
    max=0.3,
    step=0.01,
    description='Tamaño de malla (h):',
    style=style,
    layout=layout,
    readout_format='.2f'
)

# Viscosity
viscosity_widget = widgets.FloatLogSlider(
    value=0.01,
    base=10,
    min=-4,  # 0.0001
    max=-1,  # 0.1
    step=0.1,
    description='Viscosidad (ν):',
    style=style,
    layout=layout,
    readout_format='.4f'
)

reynolds_label = widgets.HTML(value="<b>Número de Reynolds: 100</b>")

def update_reynolds(change):
    re = 1.0 / viscosity_widget.value
    reynolds_label.value = f"<b>Número de Reynolds: {re:.0f}</b>"

viscosity_widget.observe(update_reynolds, names='value')

# Solver selection
solver_widget = widgets.Dropdown(
    options=[('Picard (robusto)', 'picard'),
             ('Newton (rápido)', 'newton'),
             ('Adaptativo (automático)', 'adaptive')],
    value='picard',
    description='Método:',
    style=style,
    layout=layout
)

# Tolerance
tolerance_widget = widgets.FloatLogSlider(
    value=1e-8,
    base=10,
    min=-12,
    max=-4,
    step=1,
    description='Tolerancia:',
    style=style,
    layout=layout,
    readout_format='.2e'
)

# Max iterations
max_iter_widget = widgets.IntSlider(
    value=100,
    min=10,
    max=200,
    step=10,
    description='Max iteraciones:',
    style=style,
    layout=layout
)

# Convection form (for Picard)
convection_widget = widgets.Dropdown(
    options=[('Standard', 'standard'),
             ('Divergence', 'divergence'),
             ('Skew-symmetric', 'skew_symmetric')],
    value='standard',
    description='Forma convectiva:',
    style=style,
    layout=layout
)

# Adaptive parameters
n_refinements_widget = widgets.IntSlider(
    value=5,
    min=1,
    max=10,
    step=1,
    description='Ciclos refinamiento:',
    style=style,
    layout=layout
)

theta_widget = widgets.FloatSlider(
    value=0.7,
    min=0.3,
    max=0.9,
    step=0.05,
    description='Parámetro θ:',
    style=style,
    layout=layout,
    readout_format='.2f'
)

marking_widget = widgets.Dropdown(
    options=[('Maximum', 'maximum'),
             ('Dörfler', 'dorfler')],
    value='maximum',
    description='Estrategia marcado:',
    style=style,
    layout=layout
)

# Container for solver-specific widgets
picard_newton_widgets = widgets.VBox([convection_widget, tolerance_widget, max_iter_widget])
adaptive_widgets = widgets.VBox([n_refinements_widget, theta_widget, marking_widget])

solver_params = widgets.VBox([picard_newton_widgets])

def update_solver_params(change):
    if change['new'] == 'adaptive':
        solver_params.children = [adaptive_widgets]
    else:
        if change['new'] == 'picard':
            picard_newton_widgets.children = [convection_widget, tolerance_widget, max_iter_widget]
        else:
            picard_newton_widgets.children = [tolerance_widget, max_iter_widget]
        solver_params.children = [picard_newton_widgets]

solver_widget.observe(update_solver_params, names='value')

# Display all widgets
display(HTML("<h3>Configuración del Problema</h3>"))
display(widgets.VBox([
    domain_widget,
    mesh_size_widget,
    viscosity_widget,
    reynolds_label,
    solver_widget,
    solver_params
]))

## Resolver Problema

In [ ]:
# Solve button
solve_button = widgets.Button(
    description='▶ RESOLVER',
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px'),
    style={'font_weight': 'bold'}
)

output_area = widgets.Output()

# Global variables to store results
solution_data = {'solution': None, 'info': None, 'mesh': None}

def on_solve_clicked(b):
    with output_area:
        clear_output()
        
        print("="*80)
        print(" "*30 + "RESOLVIENDO...")
        print("="*80)
        
        try:
            # Create mesh
            print("\n1. Generando malla...")
            if domain_widget.value == 'rectangle':
                mesh = rectangle_mesh(0, 1, 0, 1, h=mesh_size_widget.value)
                dirichlet = "left|right|floor"
                bc_label = "top"
            else:
                mesh = L_mesh(h=mesh_size_widget.value)
                dirichlet = "side"
                bc_label = "flow_in"
            
            print(f"   ✓ {mesh.ne} elementos, {mesh.nv} vértices")
            
            # Boundary condition
            u_bc = CF((1, 0))
            
            # Solve
            print("\n2. Resolviendo Navier-Stokes...")
            
            if solver_widget.value == 'picard':
                solution, info = picard_iteration(
                    mesh, taylor_hood(mesh, dirichlet),
                    bc_label, u_bc,
                    viscosity=viscosity_widget.value,
                    convection_form=convection_widget.value,
                    tolerance=tolerance_widget.value,
                    max_iterations=max_iter_widget.value,
                    verbose=False
                )
            elif solver_widget.value == 'newton':
                solution, info = newton_iteration(
                    mesh, taylor_hood(mesh, dirichlet),
                    bc_label, u_bc,
                    viscosity=viscosity_widget.value,
                    tolerance=tolerance_widget.value,
                    max_iterations=max_iter_widget.value,
                    verbose=False
                )
            else:  # adaptive
                solution, info = adaptive_solve(
                    mesh, bc_label, u_bc,
                    viscosity=viscosity_widget.value,
                    n_refinements=n_refinements_widget.value,
                    theta=theta_widget.value,
                    marking_strategy=marking_widget.value,
                    verbose=False
                )
            
            # Store results
            solution_data['solution'] = solution
            solution_data['info'] = info
            solution_data['mesh'] = mesh
            
            # Display results
            print("\n" + "="*80)
            print(" "*28 + "✓ SOLUCIÓN ENCONTRADA")
            print("="*80)
            
            if solver_widget.value in ['picard', 'newton']:
                print(f"\n  Convergió: {'✓ Sí' if info['converged'] else '✗ No'}")
                print(f"  Iteraciones: {info['iterations']}")
                print(f"  Error final: {info['final_error']:.6e}")
            else:
                print(f"\n  Elementos finales: {info['n_elements'][-1]}")
                print(f"  DOFs finales: {info['n_dofs'][-1]}")
                print(f"  Error final: {info['eta_global'][-1]:.6e}")
                if info['convergence_rate_eta']:
                    avg_rate = sum(info['convergence_rate_eta']) / len(info['convergence_rate_eta'])
                    print(f"  Tasa convergencia: {avg_rate:.2f}")
            
            print("\n" + "="*80)
            print("\n✓ Ejecuta la siguiente celda para visualizar los resultados")
            
        except Exception as e:
            print(f"\n❌ Error: {e}")
            import traceback
            traceback.print_exc()

solve_button.on_click(on_solve_clicked)

display(solve_button)
display(output_area)

## Visualización de Resultados

In [ ]:
# Visualization
if solution_data['solution'] is not None:
    solution = solution_data['solution']
    info = solution_data['info']
    mesh = solution_data['mesh']
    
    velocity = solution.components[0]
    pressure = solution.components[1]
    
    print("Abriendo visualizaciones...")
    Draw(velocity, mesh, "Velocidad")
    Draw(pressure, mesh, "Presión")
    
    # Plot convergence for adaptive
    if solver_widget.value == 'adaptive':
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.loglog(info['n_elements'], info['eta_global'], 'o-', linewidth=2, markersize=8)
        plt.xlabel('Número de Elementos')
        plt.ylabel('Estimador de Error')
        plt.title('Convergencia Adaptativa')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        cycles = range(1, len(info['n_elements']) + 1)
        plt.semilogy(cycles, info['n_dofs'], 's-', linewidth=2, markersize=8, color='orange')
        plt.xlabel('Ciclo de Refinamiento')
        plt.ylabel('Grados de Libertad')
        plt.title('Crecimiento de la Malla')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print("\n✓ Visualización completa")
else:
    print("❌ Primero debes resolver el problema (ejecuta la celda anterior)")

## Exportar Resultados (Opcional)

In [ ]:
# Export results (only for adaptive)
if solution_data['solution'] is not None and solver_widget.value == 'adaptive':
    from mmfem import export_results_latex
    
    export_results_latex(solution_data['info'], filename="results.tex")
    print("✓ Resultados exportados a results.tex")
else:
    if solution_data['solution'] is None:
        print("❌ Primero debes resolver el problema")
    else:
        print("ℹ Exportación disponible solo para método adaptativo")